# **"Setting Up Gurobi for Optimization Modeling"**

In [1]:
# Install Gurobi
!pip install gurobipy

# Import the Gurobi Package
import gurobipy as gp
from gurobipy import GRB


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 64.7 MB/s eta 0:00:00


# **"Employee Shift Planning and Wage Calculation Constants"**

In [2]:
# Constants
FT_EMPLOYEES = 4  # Number of full-time employees
PT_EMPLOYEES = 19 # Number of part-time employees
DAYS = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
FT_SHIFT_TIMES = ["Morning", "Evening"]  # Morning: 7am-3pm, Evening: 3pm-11pm

# Part-time shift times: each shift is 2 hours
PT_SHIFT_TIMES = ["7am-11am", "9am-1pm", "11am-3pm", "1pm-5pm", "3pm-7pm",
                  "5pm-9pm", "7pm-11pm", "9pm-11pm"]

PT_SHIFT_PERIODS = ["7am-9am", "9am-11am", "11am-1pm", "1pm-3pm",
                  "3pm-5pm", "5pm-7pm", "7pm-9pm", "9pm-11pm"]

FT_HOURLY_WAGE = 20
PT_HOURLY_WAGE = 16
FT_SHIFT_LENGTH = 8  # Full-time shift length in hours
PT_SHIFT_LENGTH = 4  # Part-time shift length in hours

# Staffing requirements for full-time employees
FT_REQ_FALL_SPRING = {"Mon": {"Morning": 2, "Evening": 2},
                      "Tue": {"Morning": 2, "Evening": 2},
                      "Wed": {"Morning": 2, "Evening": 2},
                      "Thu": {"Morning": 2, "Evening": 2},
                      "Fri": {"Morning": 1, "Evening": 1},
                      "Sat": {"Morning": 1, "Evening": 1},
                      "Sun": {"Morning": 1, "Evening": 1}}

# Staffing requirements for part-time employees for each 2-hour block
PT_REQ_FALL_SPRING_NEW = {
    "Mon": {"7am-9am": 3, "9am-11am": 3, "11am-1pm": 3, "1pm-3pm": 3,
            "3pm-5pm": 3, "5pm-7pm": 3, "7pm-9pm": 3, "9pm-11pm": 3},
    "Tue": {"7am-9am": 3, "9am-11am": 3, "11am-1pm": 3, "1pm-3pm": 3,
            "3pm-5pm": 3, "5pm-7pm": 3, "7pm-9pm": 3, "9pm-11pm": 3},
    "Wed": {"7am-9am": 3, "9am-11am": 3, "11am-1pm": 3, "1pm-3pm": 3,
            "3pm-5pm": 3, "5pm-7pm": 3, "7pm-9pm": 3, "9pm-11pm": 3},
    "Thu": {"7am-9am": 3, "9am-11am": 3, "11am-1pm": 3, "1pm-3pm": 3,
            "3pm-5pm": 3, "5pm-7pm": 3, "7pm-9pm": 3, "9pm-11pm": 3},
    "Fri": {"7am-9am": 3, "9am-11am": 3, "11am-1pm": 3, "1pm-3pm": 3,
            "3pm-5pm": 3, "5pm-7pm": 3, "7pm-9pm": 3, "9pm-11pm": 3},
    "Sat": {"7am-9am": 4, "9am-11am": 4, "11am-1pm": 4, "1pm-3pm": 4,
            "3pm-5pm": 4, "5pm-7pm": 4, "7pm-9pm": 4, "9pm-11pm": 4},
    "Sun": {"7am-9am": 4, "9am-11am": 4, "11am-1pm": 4, "1pm-3pm": 4,
            "3pm-5pm": 4, "5pm-7pm": 4, "7pm-9pm": 4, "9pm-11pm": 4}
}

# Define shift periods to manage overlapping shifts
shift_periods = {
    "7am-11am": ["7am-9am", "9am-11am"],
    "9am-1pm": ["9am-11am", "11am-1pm"],
    "11am-3pm": ["11am-1pm", "1pm-3pm"],
    "1pm-5pm": ["1pm-3pm", "3pm-5pm"],
    "3pm-7pm": ["3pm-5pm", "5pm-7pm"],
    "5pm-9pm": ["5pm-7pm", "7pm-9pm"],
    "7pm-11pm": ["7pm-9pm", "9pm-11pm"],
    "9pm-11pm": ["9pm-11pm"]
}


# **"Model Initialization and Constraints for Workforce Scheduling Optimization"**

In [3]:
# Initialize the model
model = gp.Model("Workforce_Scheduling")

# Decision Variables for full-time employees
work_ft = model.addVars(FT_EMPLOYEES, DAYS, FT_SHIFT_TIMES, vtype=GRB.BINARY, name="work_ft")

# Decision Variables for part-time employees
work_pt = model.addVars(PT_EMPLOYEES, DAYS, PT_SHIFT_TIMES, vtype=GRB.BINARY, name="work_pt")

# Objective Function: Minimize total wage costs
total_wages_ft = sum(work_ft[i, day, shift] * FT_HOURLY_WAGE * FT_SHIFT_LENGTH
                     for i in range(FT_EMPLOYEES)
                     for day in DAYS
                     for shift in FT_SHIFT_TIMES)
total_wages_pt = sum(work_pt[j, day, shift] * PT_HOURLY_WAGE * PT_SHIFT_LENGTH
                     for j in range(PT_EMPLOYEES)
                     for day in DAYS
                     for shift in PT_SHIFT_TIMES)
model.setObjective(total_wages_ft + total_wages_pt, GRB.MINIMIZE)

# Full-time staffing requirements
for day in DAYS:
    for shift in FT_SHIFT_TIMES:
        model.addConstr(sum(work_ft[i, day, shift] for i in range(FT_EMPLOYEES)) == FT_REQ_FALL_SPRING[day][shift],
                        f"ft_req_{day}_{shift}")

# Part-time staffing requirements for every 4-hour block
for day in DAYS:
    for period in PT_SHIFT_PERIODS:
        model.addConstr(
            sum(work_pt[j, day, shift]
                for j in range(PT_EMPLOYEES)
                for shift in PT_SHIFT_TIMES
                if period in shift_periods[shift]
            )
            >= PT_REQ_FALL_SPRING_NEW[day][period],
            f"pt_coverage_{day}_{shift}"
        )

# Prevent overlapping shifts for part-time employees using shift periods
for j in range(PT_EMPLOYEES):
    for day in DAYS:
        for period in PT_SHIFT_PERIODS:
            model.addConstr(
                sum(work_pt[j, day, shift]
                    for shift in PT_SHIFT_TIMES
                    if period in shift_periods[shift]
                )
                <=
                1,
                f"no_overlap_pt_{j}_{day}_{period}"
            )

# Full-time employee minimum hours constraint
for i in range(FT_EMPLOYEES):
    model.addConstr(sum(work_ft[i, day, shift] for day in DAYS for shift in FT_SHIFT_TIMES) * FT_SHIFT_LENGTH >= 40,
                    f"ft_min_hours_{i}")

# Part-time employee maximum hours constraint
for j in range(PT_EMPLOYEES):
    model.addConstr(sum(work_pt[j, day, shift] for day in DAYS for shift in PT_SHIFT_TIMES) * PT_SHIFT_LENGTH <= 20,
                    f"pt_max_hours_{j}")

Restricted license - for non-production use only - expires 2025-11-24


# **"Optimization Results: Employee Scheduling and Cost Analysis"**

In [5]:
# Optimize the model and output the results
model.optimize()

if model.status == GRB.Status.OPTIMAL:
    print("Optimal schedule found:\n")

    # Full-time employees' schedule
    print("Full-Time Employees Schedule:")
    for i in range(FT_EMPLOYEES):
        print(f"  Employee {i+1}:")
        for day in DAYS:
            shifts = [(shift, work_ft[i, day, shift].x * FT_SHIFT_LENGTH)
                      for shift in FT_SHIFT_TIMES if work_ft[i, day, shift].x > 0.5]
            shift_info = ', '.join([f"{shift} ({hours} hrs)" for shift, hours in shifts])
            print(f"    {day}: {shift_info if shift_info else 'Off'}")
        print()

    # Part-time employees' schedule
    print("Part-Time Employees Schedule:")
    for j in range(PT_EMPLOYEES):
        print(f"  Employee {j+1}:")
        for day in DAYS:
            shifts = [(shift, work_pt[j, day, shift].x * PT_SHIFT_LENGTH)
                      for shift in PT_SHIFT_TIMES if work_pt[j, day, shift].x > 0.5]
            shift_info = ', '.join([f"{shift} ({hours} hrs)" for shift, hours in shifts])
            print(f"    {day}: {shift_info if shift_info else 'Off'}")
        print()

    # Calculate and display total costs
    total_cost_ft = sum(work_ft[i, day, shift].x * FT_HOURLY_WAGE * FT_SHIFT_LENGTH
                        for i in range(FT_EMPLOYEES)
                        for day in DAYS
                        for shift in FT_SHIFT_TIMES)
    total_cost_pt = sum(work_pt[j, day, shift].x * PT_HOURLY_WAGE * PT_SHIFT_LENGTH
                        for j in range(PT_EMPLOYEES)
                        for day in DAYS
                        for shift in PT_SHIFT_TIMES)

    print(f"Total Cost for Full-Time Employees: ${total_cost_ft:.2f}")
    print(f"Total Cost for Part-Time Employees: ${total_cost_pt:.2f}\n")
    print(f"Total Cost of Optimal Schedule: ${model.ObjVal:.2f}\n")

else:
    print("No optimal solution found or model is infeasible.")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 1157 rows, 1120 columns and 5166 nonzeros
Model fingerprint: 0xdb6bc24a
Variable types: 0 continuous, 1120 integer (1120 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [6e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Presolved: 1006 rows, 1064 columns, 4921 nonzeros

Continuing optimization...


Explored 1 nodes (278 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 2 (of 2 available processors)

Solution count 1: 9408 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.408000000000e+03, best bound 9.408000000000e+03, gap 0.0000%
Optimal schedule found:

Full-Time Employees Schedule:
  Employee 1:
    Mon: Morning (8.0 hrs), Evenin